In [ ]:
from skimage import io, color, transform
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# 画像ファイルのパス
image_path = '/content/スクリーンショット 2024-11-26 134910.png'

# 画像を読み込む
image = io.imread(image_path)
if image is None:
    raise ValueError("画像が読み込めませんでした。ファイルパスを確認してください。")

# 画像をリサイズ（高速化のため小さくする場合もあり）
image = transform.resize(image, (600, 700), anti_aliasing=True)

# Check if the image has an alpha channel (4 channels)
if image.shape[2] == 4:
    # If so, remove the alpha channel by selecting only the first 3 channels (RGB)
    image = image[:, :, :3]

# 色空間をRGBからHSVに変換
hsv_image = color.rgb2hsv(image)

# 赤色と黄色の閾値を設定（色相に基づく範囲調整）
# Hueは0~1の範囲に正規化されているため、範囲を再調整
lower_red1 = (0.0, 0.4, 0.4)  # 最初の赤の範囲 (Hue, Saturation, Value)
upper_red1 = (0.05, 1.0, 1.0)
lower_red2 = (0.9, 0.4, 0.4)  # 第二の赤の範囲
upper_red2 = (1.0, 1.0, 1.0)
lower_yellow = (0.13, 0.4, 0.4)  # 黄色の範囲
upper_yellow = (0.2, 1.0, 1.0)

# マスクを作成
mask_red1 = ((hsv_image[:, :, 0] >= lower_red1[0]) & (hsv_image[:, :, 0] <= upper_red1[0]) &
             (hsv_image[:, :, 1] >= lower_red1[1]) & (hsv_image[:, :, 2] >= lower_red1[2]))
mask_red2 = ((hsv_image[:, :, 0] >= lower_red2[0]) & (hsv_image[:, :, 0] <= upper_red2[0]) &
             (hsv_image[:, :, 1] >= lower_red2[1]) & (hsv_image[:, :, 2] >= lower_red2[2]))
mask_red = mask_red1 | mask_red2

mask_yellow = ((hsv_image[:, :, 0] >= lower_yellow[0]) & (hsv_image[:, :, 0] <= upper_yellow[0]) &
               (hsv_image[:, :, 1] >= lower_yellow[1]) & (hsv_image[:, :, 2] >= lower_yellow[2]))

# マスクの視覚化
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(mask_red, cmap='Reds')
plt.title("Red Mask")

plt.subplot(1, 2, 2)
plt.imshow(mask_yellow, cmap='YlOrBr')
plt.title("Yellow Mask")

plt.tight_layout()
plt.show()


# マスクのデバッグ表示
plt.figure(figsize=(12, 6))
plt.subplot(1, 3, 1)
plt.title("Original Image")
plt.imshow(image)

plt.subplot(1, 3, 2)
plt.title("Red Mask")
plt.imshow(mask_red, cmap='Reds')

plt.subplot(1, 3, 3)
plt.title("Yellow Mask")
plt.imshow(mask_yellow, cmap='Oranges')
plt.show()

# グリッドに分割して駒を認識
grid_rows, grid_cols = 6, 7
cell_height = image.shape[0] // grid_rows
cell_width = image.shape[1] // grid_cols

board_state = []
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image)


for row in range(grid_rows):
    row_state = []
    for col in range(grid_cols):
        y_start, y_end = row * cell_height, (row + 1) * cell_height
        x_start, x_end = col * cell_width, (col + 1) * cell_width

        red_density = np.mean(mask_red[y_start:y_end, x_start:x_end])
        yellow_density = np.mean(mask_yellow[y_start:y_end, x_start:x_end])

        if red_density > yellow_density and red_density > 0.2:  # 赤が支配的
            row_state.append('R')
            color = 'red'
        elif yellow_density > red_density and yellow_density > 0.2:  # 黄が支配的
            row_state.append('Y')
            color = 'yellow'
        else:  # 空白
            row_state.append('-')
            color = 'white'

        # 描画
        rect = Rectangle((x_start, y_start), cell_width, cell_height, linewidth=1, edgecolor='green', facecolor='none')
        ax.add_patch(rect)
        ax.text(x_start + cell_width // 3, y_start + cell_height // 2, row_state[-1], fontsize=12, color=color)
    board_state.append(row_state)

plt.axis('off')
plt.show()

# 結果の出力
output = "\n".join(" ".join(row) for row in board_state)
print(output)
